## 1. Requirements

In [1]:
import os
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim

## 2. Set Args

In [2]:
theta = 7
num_epochs = 500
dropout_ratio = 0.5

data_path = 'data/BostonHousing.csv'
mechanism = 'mcar'
method = 'uniform'

test_size = 0.3
use_cuda = True
batch_size  = 1 # not in the paper

## 3. Prepare Data

In [3]:
data = pd.read_csv(data_path).values

rows, cols = data.shape
shuffled_index = np.random.permutation(rows)
train_index = shuffled_index[:int(rows*(1-test_size))]
test_index = shuffled_index[int(rows*(1-test_size)):]

train_data = data[train_index, :]
test_data = data[test_index, :]

# standardized between 0 and 1
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [4]:
def missing_method(raw_data, mechanism='mcar', method='uniform') :
    
    data = raw_data.copy()
    rows, cols = data.shape
    
    # missingness threshold
    t = 0.2
    
    if mechanism == 'mcar' :
    
        if method == 'uniform' :
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where v<=t
            mask = (v<=t)
            data[mask] = 0

        elif method == 'random' :
            # only half of the attributes to have missing value
            missing_cols = np.random.choice(cols, cols//2)
            c = np.zeros(cols, dtype=bool)
            c[missing_cols] = True

            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where v<=t
            mask = (v<=t)*c
            data[mask] = 0

        else :
            print("Error : There are no such method")
            raise
    
    elif mechanism == 'mnar' :
        
        if method == 'uniform' :
            # randomly sample two attributes
            sample_cols = np.random.choice(cols, 2)

            # calculate ther median m1, m2
            m1, m2 = np.median(data[:,sample_cols], axis=0)
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = data[:,sample_cols[0]] <= m1
            m2 = data[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]

            mask = m*(v<=t)
            data[mask] = 0


        elif method == 'random' :
            # only half of the attributes to have missing value
            missing_cols = np.random.choice(cols, cols//2)
            c = np.zeros(cols, dtype=bool)
            c[missing_cols] = True

            # randomly sample two attributes
            sample_cols = np.random.choice(cols, 2)

            # calculate ther median m1, m2
            m1, m2 = np.median(data[:,sample_cols], axis=0)
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = data[:,sample_cols[0]] <= m1
            m2 = data[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]

            mask = m*(v<=t)*c
            data[mask] = 0

        else :
            print("Error : There is no such method")
            raise
    
    else :
        print("Error : There is no such mechanism")
        raise
        
    return data, mask

In [5]:
missed_data, mask = missing_method(test_data, mechanism=mechanism, method=method)

missed_data = torch.from_numpy(missed_data).float()
train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

## 4. Define Model

In [6]:
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
class Autoencoder(nn.Module):
    def __init__(self, dim):
        super(Autoencoder, self).__init__()
        self.dim = dim
        
        self.drop_out = nn.Dropout(p=0.5)
        
        self.encoder = nn.Sequential(
            nn.Linear(dim+theta*0, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*3)
        )
            
        self.decoder = nn.Sequential(
            nn.Linear(dim+theta*3, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*0)
        )
        
    def forward(self, x):
        x = x.view(-1, self.dim)
        x_missed = self.drop_out(x)
        
        z = self.encoder(x_missed)
        out = self.decoder(z)
        
        out = out.view(-1, self.dim)
        
        return out

In [8]:
model = Autoencoder(dim=cols).to(device)

## 5. Define Loss and Optimizer

In [9]:
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), momentum=0.99, lr=0.01, nesterov=True)

## 6. Train Model

In [10]:
cost_list = []
early_stop = False

for epoch in range(num_epochs):
    
    total_batch = len(train_data) // batch_size
    
    for i, batch_data in enumerate(train_loader):
        
        batch_data = batch_data.to(device)
        
        reconst_data = model(batch_data)
        cost = loss(reconst_data, batch_data)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
                
        if (i+1) % (total_batch//2) == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.6f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
            
        # early stopping rule 1 : MSE < 1e-06
        if cost.item() < 1e-06 :
            early_stop = True
            break
            
#         early stopping rule 2 : simple moving average of length 5
#         sometimes it doesn't work well.
#         if len(cost_list) > 5 :
#            if cost.item() > np.mean(cost_list[-5:]):
#                early_stop = True
#                break
                
        cost_list.append(cost.item())

    if early_stop :
        break
        
print("Learning Finished!")

Epoch [1/500], lter [177/354], Loss: 0.064384
Epoch [1/500], lter [354/354], Loss: 0.056846
Epoch [2/500], lter [177/354], Loss: 0.054899
Epoch [2/500], lter [354/354], Loss: 0.033222
Epoch [3/500], lter [177/354], Loss: 0.021640
Epoch [3/500], lter [354/354], Loss: 0.020006
Epoch [4/500], lter [177/354], Loss: 0.139684
Epoch [4/500], lter [354/354], Loss: 0.031991
Epoch [5/500], lter [177/354], Loss: 0.044147
Epoch [5/500], lter [354/354], Loss: 0.078869
Epoch [6/500], lter [177/354], Loss: 0.022894
Epoch [6/500], lter [354/354], Loss: 0.074245
Epoch [7/500], lter [177/354], Loss: 0.167476
Epoch [7/500], lter [354/354], Loss: 0.057272
Epoch [8/500], lter [177/354], Loss: 0.079521
Epoch [8/500], lter [354/354], Loss: 0.050776
Epoch [9/500], lter [177/354], Loss: 0.028537
Epoch [9/500], lter [354/354], Loss: 0.018806
Epoch [10/500], lter [177/354], Loss: 0.068195
Epoch [10/500], lter [354/354], Loss: 0.011419
Epoch [11/500], lter [177/354], Loss: 0.017908
Epoch [11/500], lter [354/354],

Epoch [88/500], lter [354/354], Loss: 0.016583
Epoch [89/500], lter [177/354], Loss: 0.096540
Epoch [89/500], lter [354/354], Loss: 0.012086
Epoch [90/500], lter [177/354], Loss: 0.013446
Epoch [90/500], lter [354/354], Loss: 0.015077
Epoch [91/500], lter [177/354], Loss: 0.040354
Epoch [91/500], lter [354/354], Loss: 0.013960
Epoch [92/500], lter [177/354], Loss: 0.077843
Epoch [92/500], lter [354/354], Loss: 0.072892
Epoch [93/500], lter [177/354], Loss: 0.086571
Epoch [93/500], lter [354/354], Loss: 0.032262
Epoch [94/500], lter [177/354], Loss: 0.009919
Epoch [94/500], lter [354/354], Loss: 0.051995
Epoch [95/500], lter [177/354], Loss: 0.024357
Epoch [95/500], lter [354/354], Loss: 0.019331
Epoch [96/500], lter [177/354], Loss: 0.016462
Epoch [96/500], lter [354/354], Loss: 0.008812
Epoch [97/500], lter [177/354], Loss: 0.010586
Epoch [97/500], lter [354/354], Loss: 0.085507
Epoch [98/500], lter [177/354], Loss: 0.008209
Epoch [98/500], lter [354/354], Loss: 0.084329
Epoch [99/500

Epoch [174/500], lter [354/354], Loss: 0.018363
Epoch [175/500], lter [177/354], Loss: 0.012584
Epoch [175/500], lter [354/354], Loss: 0.007947
Epoch [176/500], lter [177/354], Loss: 0.022341
Epoch [176/500], lter [354/354], Loss: 0.014584
Epoch [177/500], lter [177/354], Loss: 0.017245
Epoch [177/500], lter [354/354], Loss: 0.006471
Epoch [178/500], lter [177/354], Loss: 0.017742
Epoch [178/500], lter [354/354], Loss: 0.068118
Epoch [179/500], lter [177/354], Loss: 0.019092
Epoch [179/500], lter [354/354], Loss: 0.022993
Epoch [180/500], lter [177/354], Loss: 0.013789
Epoch [180/500], lter [354/354], Loss: 0.019468
Epoch [181/500], lter [177/354], Loss: 0.010613
Epoch [181/500], lter [354/354], Loss: 0.007876
Epoch [182/500], lter [177/354], Loss: 0.003555
Epoch [182/500], lter [354/354], Loss: 0.023834
Epoch [183/500], lter [177/354], Loss: 0.032061
Epoch [183/500], lter [354/354], Loss: 0.015184
Epoch [184/500], lter [177/354], Loss: 0.043588
Epoch [184/500], lter [354/354], Loss: 0

Epoch [260/500], lter [177/354], Loss: 0.008432
Epoch [260/500], lter [354/354], Loss: 0.017943
Epoch [261/500], lter [177/354], Loss: 0.047107
Epoch [261/500], lter [354/354], Loss: 0.003890
Epoch [262/500], lter [177/354], Loss: 0.090581
Epoch [262/500], lter [354/354], Loss: 0.069404
Epoch [263/500], lter [177/354], Loss: 0.012569
Epoch [263/500], lter [354/354], Loss: 0.005013
Epoch [264/500], lter [177/354], Loss: 0.042927
Epoch [264/500], lter [354/354], Loss: 0.009988
Epoch [265/500], lter [177/354], Loss: 0.004792
Epoch [265/500], lter [354/354], Loss: 0.097724
Epoch [266/500], lter [177/354], Loss: 0.008827
Epoch [266/500], lter [354/354], Loss: 0.026646
Epoch [267/500], lter [177/354], Loss: 0.043402
Epoch [267/500], lter [354/354], Loss: 0.031314
Epoch [268/500], lter [177/354], Loss: 0.023740
Epoch [268/500], lter [354/354], Loss: 0.012221
Epoch [269/500], lter [177/354], Loss: 0.025098
Epoch [269/500], lter [354/354], Loss: 0.012924
Epoch [270/500], lter [177/354], Loss: 0

Epoch [345/500], lter [354/354], Loss: 0.008919
Epoch [346/500], lter [177/354], Loss: 0.037060
Epoch [346/500], lter [354/354], Loss: 0.015137
Epoch [347/500], lter [177/354], Loss: 0.037601
Epoch [347/500], lter [354/354], Loss: 0.034827
Epoch [348/500], lter [177/354], Loss: 0.008614
Epoch [348/500], lter [354/354], Loss: 0.013820
Epoch [349/500], lter [177/354], Loss: 0.035439
Epoch [349/500], lter [354/354], Loss: 0.019931
Epoch [350/500], lter [177/354], Loss: 0.017511
Epoch [350/500], lter [354/354], Loss: 0.037929
Epoch [351/500], lter [177/354], Loss: 0.003026
Epoch [351/500], lter [354/354], Loss: 0.027777
Epoch [352/500], lter [177/354], Loss: 0.014957
Epoch [352/500], lter [354/354], Loss: 0.010777
Epoch [353/500], lter [177/354], Loss: 0.013056
Epoch [353/500], lter [354/354], Loss: 0.007225
Epoch [354/500], lter [177/354], Loss: 0.008070
Epoch [354/500], lter [354/354], Loss: 0.004495
Epoch [355/500], lter [177/354], Loss: 0.009195
Epoch [355/500], lter [354/354], Loss: 0

Epoch [431/500], lter [177/354], Loss: 0.017543
Epoch [431/500], lter [354/354], Loss: 0.059729
Epoch [432/500], lter [177/354], Loss: 0.026644
Epoch [432/500], lter [354/354], Loss: 0.009100
Epoch [433/500], lter [177/354], Loss: 0.021734
Epoch [433/500], lter [354/354], Loss: 0.025194
Epoch [434/500], lter [177/354], Loss: 0.027408
Epoch [434/500], lter [354/354], Loss: 0.058511
Epoch [435/500], lter [177/354], Loss: 0.022110
Epoch [435/500], lter [354/354], Loss: 0.039793
Epoch [436/500], lter [177/354], Loss: 0.019929
Epoch [436/500], lter [354/354], Loss: 0.051107
Epoch [437/500], lter [177/354], Loss: 0.018474
Epoch [437/500], lter [354/354], Loss: 0.027441
Epoch [438/500], lter [177/354], Loss: 0.027411
Epoch [438/500], lter [354/354], Loss: 0.045378
Epoch [439/500], lter [177/354], Loss: 0.026342
Epoch [439/500], lter [354/354], Loss: 0.056116
Epoch [440/500], lter [177/354], Loss: 0.010557
Epoch [440/500], lter [354/354], Loss: 0.015070
Epoch [441/500], lter [177/354], Loss: 0

## 7. Test Model

In [11]:
model.eval()
filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = 0

for i in range(cols) :
    if mask[:,i].sum() > 0 :
        y_actual = test_data[:,i][mask[:,i]]
        y_predicted = filled_data[:,i][mask[:,i]]

        rmse = sqrt(mean_squared_error(y_actual, y_predicted))
        rmse_sum += rmse
    
print("RMSE_SUM :", rmse_sum)

RMSE_SUM : 3.1379940845893466
